<a href="https://colab.research.google.com/github/Viroslav/SBERMEDAI/blob/main/SbermedAI(reshaped).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import random
import numpy as np
import cv2
import os

torch.backends.cudnn.deterministic = True
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision import datasets 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision
from skimage import io
import pandas as pd
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score



# fix random seed
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)

# using cuda
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
class AnginaNotAngina(Dataset):
    def __init__(self, xlsx_file, root_dir, transform = None):
        self.annotations = pd.read_excel(xlsx_file, index_col=0)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        
        if self.transform:
            image = self.transform(image)
            
        return(image, y_label)

In [3]:
# reshape for GoogleNet
down_width = 224

down_height = 224

down_points = (down_width, down_height)


In [4]:
count = 1
while count < 44:
  image = cv2.imread(f'/content/Images/ang{count}.jpg')
  image_resized = cv2.resize(image, down_points, interpolation= cv2.INTER_LINEAR)
  cv2.imwrite("/content/resized/ang%d.jpg" % count, image_resized)
  count += 1

In [5]:
dataset = AnginaNotAngina(xlsx_file = '/content/labels.xlsx',
                          root_dir = '/content/resized',
                          transform = transforms.ToTensor())

In [6]:
len(dataset)

43

In [7]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

In [8]:
batch_size = 1

In [9]:
train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle = True)

In [10]:
train_loader

## Using GoogleNet

In [11]:
model = torchvision.models.googlenet(pretrained = True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

## Optimizer

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [14]:
num_epochs = 4

## Traininng

In [15]:
for epoch in range(num_epochs):
    losses = []
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device = device)
        targets = targets.to(device = device)
        
        #forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        losses.append(loss.item())
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #adam step
        optimizer.step()
    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

Cost at epoch 0 is 1.8757098361849784
Cost at epoch 1 is 0.5082450740039348
Cost at epoch 2 is 0.31493966657047473
Cost at epoch 3 is 0.16484217920030156
Cost at epoch 4 is 0.04399504555234065


## Accuracy

In [16]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    y_true = []
    y_score = []
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            y_true.append(y.item())
            y_score.append(predictions.item())
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}", '\t',  
              f"and roc_auc_score: {roc_auc_score(y_true, y_score)} ")

    model.train()

In [17]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

Checking accuracy on Training Set
Got 30 / 30 with accuracy 100.0 	 and roc_auc_score: 1.0 


In [18]:
print("Checking accuracy on Testing Set")
check_accuracy(test_loader, model)

Checking accuracy on Testing Set
Got 9 / 13 with accuracy 69.23076923076923 	 and roc_auc_score: 0.7125 
